In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep

In [3]:
# driver = webdriver.Firefox(executable_path = '/usr/local/bin/geckodriver')
driver = webdriver.Chrome(executable_path = '/usr/local/bin/chromedriver')  # Optional argument, if not specified will search path.

In [4]:
driver.get("https://www.netflix.com/")
login = driver.find_element_by_css_selector(".authLinks")
login.click()

In [5]:
element = driver.find_element_by_name("userLoginId")
element.clear()
element.send_keys("johndoe@gmail.com")
element2 = driver.find_element_by_name("password")
element2.clear()
element2.send_keys("qwertyuiop")

In [6]:
submit = driver.find_element_by_css_selector(".btn.login-button.btn-submit.btn-small")
submit.click()

In [7]:
# change profile
# driver.find_element_by_css_selector('.profile-name[text()="Sachit"]').click()
sleep(3)
driver.find_element_by_xpath("//span[@class='profile-name' and text()='Sachit']").click()

In [8]:
def searchMechanism(keyword, inputClose = True):
    if(inputClose):
        searchButton = driver.find_element_by_css_selector(".searchTab")
        searchButton.click()
    searchInput = driver.find_element_by_name("searchInput")
    searchInput.clear()
    searchInput.send_keys(keyword)

In [18]:
def search(keyword):
    driver.get("https://www.netflix.com/")
    sleep(5)
    try:
        searchInput = driver.find_element_by_name("searchInput")
    except Exception as ex:
        if(ex.__class__.__name__ == 'NoSuchElementException'):
            searchMechanism(keyword)
    finally:
        searchMechanism(keyword, False)
        sleep(3)
        firstResult = driver.find_element_by_css_selector(".slider-refocus")
        firstResult.click()

In [19]:
def episodeInfo(prefix, text): 
    epiArr = text.split(': ')
    season = int(epiArr[0])
    return {'seasonInfo': f'{prefix} {season}', 'season': season, 'episode': epiArr[1]}

In [20]:
def showText(text):
    showArray = text.split(': Part ')
    if(len(showArray) > 1):
        print(showArray)
        return showArray[0], episodeInfo('Part', showArray[1])
    showArray = text.split(': Season ')
    if(len(showArray) > 1):
        print(showArray)
        return showArray[0], episodeInfo('Season', showArray[1])
    return text, False

In [31]:
def moveToLast():
    sleep(5)
    player = driver.find_element_by_css_selector(".AkiraPlayer")
    sleep(5)
    player.click()
    e = driver.find_element_by_css_selector(".scrubber-bar")
    location = e.location
    size = e.size

    x = size['width'] + location['x']
    y = int(size['height']/2) + location['y']

    sleep(5)
    player.click()
    actions = webdriver.ActionChains(driver)
    actions.move_to_element_with_offset(player, x, y).click().perform()

In [32]:
def findEpisode(episodeName, expandedList = False):
    try:
        episode = driver.find_element_by_xpath(f'//span[@class="titleCard-title_text" and text()="{episodeName}"]')
        episode.click()
    except Exception as ex:
        if(ex.__class__.__name__ == 'NoSuchElementException'):
            if not expandedList:
                driver.find_element_by_css_selector(".section-expandButton").click()
            findEpisode(episodeName, True)
            

In [33]:
def performAction(searchKey):
    try:
        show, seriesInfo = showText(searchKey)
        print(show, seriesInfo)
        search(show)
        if seriesInfo:
            sleep(3)
            dropdown = driver.find_element_by_css_selector(".dropdown-toggle")
            seasonInfo = seriesInfo['seasonInfo']
            currentSE = dropdown.text
            print(currentSE, seasonInfo)
            if currentSE != seasonInfo:
                dropdown.click()
                driver.find_element_by_xpath(f'//li[@data-uia="dropdown-menu-item" and text()="{seasonInfo}"]').click()
            findEpisode(seriesInfo['episode'])
        else:
            print('playing movie')
            driver.find_element_by_css_selector(".playLink").click()
    except Exception as ex:
        if(ex.__class__.__name__ == 'NoSuchElementException'):
            print(ex)
            driver.find_element_by_css_selector(".playLink").click()
            
    print('moving to last')
    moveToLast()

In [34]:
import csv
showList = []
with open('./NetflixViewingHistory.csv', newline='') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
#     for row, index in spamreader:
#         print(row)
     reader = csv.DictReader(csvfile)
     for row in reader:
        showList.append(row['Title'])

In [35]:
for showFullName in reversed(showList):
    performAction(showFullName)

['The Witcher', '1: The End’s Beginning']
The Witcher {'seasonInfo': 'Season 1', 'season': 1, 'episode': 'The End’s Beginning'}
Message: no such element: Unable to locate element: {"method":"css selector","selector":".dropdown-toggle"}
  (Session info: chrome=86.0.4240.111)

moving to last
['Lucifer', '1: Pilot']
Lucifer {'seasonInfo': 'Season 1', 'season': 1, 'episode': 'Pilot'}
Season 1 Season 1
moving to last
['Lucifer', '1: Lucifer, Stay. Good Devil.']
Lucifer {'seasonInfo': 'Season 1', 'season': 1, 'episode': 'Lucifer, Stay. Good Devil.'}
Season 1 Season 1
moving to last
['Lucifer', '1: The Would-Be Prince of Darkness']
Lucifer {'seasonInfo': 'Season 1', 'season': 1, 'episode': 'The Would-Be Prince of Darkness'}
Season 1 Season 1
moving to last
['Lucifer', '1: Manly Whatnots']
Lucifer {'seasonInfo': 'Season 1', 'season': 1, 'episode': 'Manly Whatnots'}
Season 1 Season 1
moving to last
['Lucifer', '1: Sweet Kicks']
Lucifer {'seasonInfo': 'Season 1', 'season': 1, 'episode': 'Sweet K

In [ ]:
# driver.close()